# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## PREPARE TRAIN AND TEST DATASET

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [11]:
from datasets import load_dataset

# Load the Amazon Reviews dataset
dataset = load_dataset("sms_spam",split="train").train_test_split(
                        test_size=0.2,shuffle=True,seed=23
)

# Print dataset information
print(dataset)

splits=["train","test"]

print(dataset["train"])
print(dataset["train"][0])
print(dataset["test"])
print(dataset["test"][0])

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 1115
    })
})
Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})
{'sms': 'Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call\n', 'label': 1}
Dataset({
    features: ['sms', 'label'],
    num_rows: 1115
})
{'sms': 'Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n', 'label': 0}


In [ ]:
## TOKENIZE TEST AND TRAIN DATASET

In [12]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
print(tokenizer("Hello world"))
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to end-of-sequence token

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True, padding=True, return_tensors="pt"), batched=True
    )

# Inspect the available columns in the dataset
print(tokenized_dataset["train"].column_names)
train_lora = tokenized_dataset['train'].rename_column('label', 'labels')
test_lora = tokenized_dataset['test'].rename_column('label', 'labels')
print(train_lora[0])

{'input_ids': [15496, 995], 'attention_mask': [1, 1]}
['sms', 'label', 'input_ids', 'attention_mask']
{'sms': 'Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call\n', 'labels': 1, 'input_ids': [25383, 534, 5175, 838, 285, 9998, 30, 10133, 284, 262, 3452, 20432, 14, 10798, 9512, 329, 17189, 13, 509, 35238, 37902, 311, 10067, 36871, 13246, 11, 3497, 3131, 1479, 23550, 14, 5239, 82, 13, 8255, 21560, 329, 257, 869, 198, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1,

In [7]:
#LOAD A PRE-TRAINED MODEL(GPT2 IN THIS CASE)
import torch
import pandas as pd
from transformers import AutoModelForSequenceClassification
model2 = AutoModelForSequenceClassification.from_pretrained("gpt2",num_labels=2,id2label={0: "not spam",1: "spam"},label2id={"not spam": 0, "spam": 1})


from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)  # Example with 2 labels
model.config.pad_token_id = model.config.eos_token_id
input_data=test_lora.select([0,1,22,31,43,292,448,487])
# Initialize empty lists to store predictions
predicted_classes = []
predicted_probabilities = []

# Iterate through input data
for data_row in input_data:
    # Prepare input tensors
    input_ids = torch.tensor([data_row['input_ids']])
    attention_mask = torch.tensor([data_row['attention_mask']])

    # Forward pass
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Get predicted probabilities and class
    probabilities = torch.softmax(outputs.logits, dim=-1).tolist()[0]
    predicted_class = torch.argmax(torch.tensor(probabilities), dim=-1).item()

    predicted_classes.append(predicted_class)
    predicted_probabilities.append(probabilities)

# Print results

df=pd.DataFrame(
    {
        "sms":[item["sms"] for item in input_data],
        "predictions": predicted_classes,
        "labels": [item["labels"] for item in input_data] ,
    
    }
)
pd.set_option("display.max_colwidth",None)
df

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,1,0
1,Happy new years melody!\n,1,0
2,PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S. I. M. points. Call 08715203652 Identifier Code: 42810 Expires 29/10/0\n,1,1
3,URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid 12hrs only\n,1,1
4,I had askd u a question some hours before. Its answer\n,1,0
5,"SMS. ac JSco: Energy is high, but u may not know where 2channel it. 2day ur leadership skills r strong. Psychic? Reply ANS w/question. End? Reply END JSCO\n",1,1
6,"Yun ah.the ubi one say if ü wan call by tomorrow.call 67441233 look for irene.ere only got bus8,22,65,61,66,382. Ubi cres,ubi tech park.6ph for 1st 5wkg days.èn\n",1,0
7,Burger King - Wanna play footy at a top stadium? Get 2 Burger King before 1st Sept and go Large or Super with Coca-Cola and walk out a winner\n,1,1


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [13]:
from peft import LoraConfig
from peft import get_peft_model
from transformers import AutoModelForSequenceClassification
model2 = AutoModelForSequenceClassification.from_pretrained("gpt2",num_labels=2,id2label={0: "not spam",1: "spam"},label2id={"not spam": 0, "spam": 1})
model = AutoModelForSequenceClassification.from_pretrained("gpt2")
model2.config.pad_token_id = model.config.eos_token_id
config = LoraConfig(
r=8, 
lora_alpha=32,
target_modules=['c_attn', 'c_proj'],
lora_dropout=0.1,
bias="none",
fan_in_fan_out=True,
task_type="SEQ_CLS"
)
lora_model = get_peft_model(model2, config)
lora_model.print_trainable_parameters()
print(lora_model)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 814,080 || all params: 125,253,888 || trainable%: 0.6499438963523432
PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
               

In [14]:

import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=6,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=train_lora,
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.117787,0.973094
2,0.228900,0.072773,0.987444
3,0.228900,0.072724,0.987444
4,0.084900,0.069218,0.988341
5,0.084900,0.070712,0.987444
6,0.063700,0.069418,0.988341


Checkpoint destination directory ./data/spam_not_spam/checkpoint-279 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/spam_not_spam/checkpoint-558 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/spam_not_spam/checkpoint-837 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1674, training_loss=0.12014915438963093, metrics={'train_runtime': 1269.4181, 'train_samples_per_second': 21.076, 'train_steps_per_second': 1.319, 'total_flos': 3536467192074240.0, 'train_loss': 0.12014915438963093, 'epoch': 6.0})

In [5]:
# Show the performance of the model on the test set
# What do you think the evaluation accuracy will be?
trainer.evaluate()

{'eval_loss': 0.052338458597660065,
 'eval_accuracy': 0.9865470852017937,
 'eval_runtime': 16.4687,
 'eval_samples_per_second': 67.704,
 'eval_steps_per_second': 4.25,
 'epoch': 3.0}

In [15]:
lora_model.save_pretrained("./data/model")

In [16]:
from transformers import AutoModelForSequenceClassification
import torch
import pandas as pd

model = AutoModelForSequenceClassification.from_pretrained("./data/model",num_labels=2,id2label={0: "not spam",1: "spam"},label2id={"not spam": 0, "spam": 1})
model.config.pad_token_id = model.config.eos_token_id
input_data=test_lora.select([0,1,22,31,43,292,448,487])
# Initialize empty lists to store predictions
predicted_classes = []
predicted_probabilities = []

# Iterate through input data
for data_row in input_data:
    # Prepare input tensors
    input_ids = torch.tensor([data_row['input_ids']])
    attention_mask = torch.tensor([data_row['attention_mask']])

    # Forward pass
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Get predicted probabilities and class
    probabilities = torch.softmax(outputs.logits, dim=-1).tolist()[0]
    predicted_class = torch.argmax(torch.tensor(probabilities), dim=-1).item()

    predicted_classes.append(predicted_class)
    predicted_probabilities.append(probabilities)

# Print results

df=pd.DataFrame(
    {
        "sms":[item["sms"] for item in input_data],
        "predictions": predicted_classes,
        "labels": [item["labels"] for item in input_data] ,
    
    }
)
pd.set_option("display.max_colwidth",None)
df

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,0,0
1,Happy new years melody!\n,0,0
2,PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S. I. M. points. Call 08715203652 Identifier Code: 42810 Expires 29/10/0\n,1,1
3,URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid 12hrs only\n,1,1
4,I had askd u a question some hours before. Its answer\n,0,0
5,"SMS. ac JSco: Energy is high, but u may not know where 2channel it. 2day ur leadership skills r strong. Psychic? Reply ANS w/question. End? Reply END JSCO\n",0,1
6,"Yun ah.the ubi one say if ü wan call by tomorrow.call 67441233 look for irene.ere only got bus8,22,65,61,66,382. Ubi cres,ubi tech park.6ph for 1st 5wkg days.èn\n",0,0
7,Burger King - Wanna play footy at a top stadium? Get 2 Burger King before 1st Sept and go Large or Super with Coca-Cola and walk out a winner\n,0,1


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,0,0
1,Happy new years melody!\n,0,0
2,PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S. I. M. points. Call 08715203652 Identifier Code: 42810 Expires 29/10/0\n,1,1
3,URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid 12hrs only\n,1,1
4,I had askd u a question some hours before. Its answer\n,0,0
5,"SMS. ac JSco: Energy is high, but u may not know where 2channel it. 2day ur leadership skills r strong. Psychic? Reply ANS w/question. End? Reply END JSCO\n",0,1
6,"Yun ah.the ubi one say if ü wan call by tomorrow.call 67441233 look for irene.ere only got bus8,22,65,61,66,382. Ubi cres,ubi tech park.6ph for 1st 5wkg days.èn\n",1,0
7,Burger King - Wanna play footy at a top stadium? Get 2 Burger King before 1st Sept and go Large or Super with Coca-Cola and walk out a winner\n,0,1
